# <span style="color:green"><center>Diplomado en Big Data</center></span>

# <span style="color:red"><center> Dask Distributed: Ejecución distribuida<center></span>

<img src="../images/dask_horizontal.svg" align="right" width="30%">


##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Contenido</span>


* [Introducción](#Introducción)


## <span style="color:blue">Fuente</span>

Esta es una traducción libre del tutorial disponible en [dask-tutorial](https://github.com/dask/dask-tutorial).

## <span style="color:blue">Introducción</span>

Como hemos visto hasta ahora, Dask le permite simplemente construir grafos de tareas con dependencias, así como tener grafos creados automáticamente para usted usando sintaxis funcional con Numpy o Pandas en colecciones de datos. Nada de esto sería muy útil, si no hubiera también una forma de ejecutar estos grafos, de forma paralela y consciente de la memoria. Hasta ahora hemos estado llamando a `thing.compute()` o `dask.compute(thing)` sin preocuparnos de lo que esto implica. Ahora discutiremos las opciones disponibles para esa ejecución, y en particular, el programador distribuido (`distributed scheduler`), que viene con funcionalidad adicional.

Dask viene con cuatro programadores disponibles:

- "threaded" (aka "threading"): un programador respaldado por un grupo de subprocesos
- "processes": un programador respaldado por un grupo de procesos
- "single-threaded" (aka "sync"): Un planificador síncrono, bueno para la depuración
- distributed: un programador distribuido para ejecutar grafos en varias máquinas, consulte abajo.

Para seleccionar uno de estos para el cálculo, puede especificarlo en el momento de solicitar un resultado, p. Ej.,
```python
myvalue.compute(scheduler="single-threaded")  # for debugging
```

También puede configurar un programador predeterminado ya sea temporalmente
```python
with dask.config.set(scheduler='processes'):
    # set temporarily for this block only
    # all compute calls within this block will use the specified scheduler
    myvalue.compute()
    anothervalue.compute()
```

O globalmente
```python
# set until further notice
dask.config.set(scheduler='processes')
```

Probemos algunos programadores en el caso familiar de los datos de vuelos.

In [ ]:
%run prep.py -d flights

In [5]:
import dask.dataframe as dd
import os
df = dd.read_csv(os.path.join('../data/', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': object,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})

# Maximum average non-cancelled delay grouped by Airport
largest_delay = df[~df.Cancelled].groupby('Origin').DepDelay.mean().max()
largest_delay

dd.Scalar<series-..., dtype=float64>

In [2]:
# each of the following gives the same results (you can check!)
# any surprises?
import time
for sch in ['threading', 'processes', 'sync']:
    t0 = time.time()
    r = largest_delay.compute(scheduler=sch)
    t1 = time.time()
    print(f"{sch:>10}, {t1 - t0:0.4f} s; result, {r:0.2f} hours")

 threading, 9.5240 s; result, 10.35 hours
 processes, 19.3503 s; result, 10.35 hours
      sync, 10.4881 s; result, 10.35 hours


### Algunas preguntas a considerar:

- Cuánta aceleración es posible para esta tarea (pista, revise el grafo).
- Teniendo en cuenta la cantidad de núcleos que hay en esta máquina, cuánto más rápidos podrían ser los programadores paralelos comparados con el programador de un solo subproceso.
- ¿Cuánto más rápido fue usar hilos en un solo hilo? ¿Por qué esto difiere de la aceleración óptima?
- ¿Por qué el planificador de multiprocesamiento es mucho más lento aquí?

El programador `threaded` es una buena elección para trabajar con grandes conjuntos de datos fuera del núcleo en una sola máquina, siempre que las funciones que se utilizan liberen [GIL](https://blockgeni.com/tutorial-python-global-interpreter-lock/) la mayor parte del tiempo. NumPy y Pandas liberan el GIL en la mayoría de los lugares, por lo que el programador `threaded` es el predeterminado para `dask.array` y `dask.dataframe`. El planificador distribuido, tal vez con `process = False`, también funcionará bien para estas cargas de trabajo en una sola máquina.

Para cargas de trabajo que contienen GIL, como es común con `dask.bag` y el código personalizado envuelto con `dask.delayed`, se recomienda usar el programador distribuido, incluso en una sola máquina. En términos generales, es más inteligente y proporciona mejores diagnósticos que el programador de "procesos".
https://docs.dask.org/en/latest/scheduling.html proporciona algunos detalles adicionales sobre la elección de un planificador.

Para escalar el trabajo en un clúster, se requiere el programador distribuido.

## Construyendo un cluster

### Método simple

El sistema `dask.distributed` se compone de un único planificador centralizado y uno o más procesos de trabajo. [Implementar](https://docs.dask.org/en/latest/setup.html) un clúster Dask remoto implica un esfuerzo adicional. Pero hacer cosas localmente solo implica crear un objeto `Client`, que le permite interactuar con el" clúster "(subprocesos o procesos locales en su máquina). Para obtener más información, consulte [aquí](https://docs.dask.org/en/latest/setup/single-distributed.html).

Tenga en cuenta que `Client()` toma muchos [argumentos](https://distributed.dask.org/en/latest/local-cluster.html#api)  opcionales , para configurar el número de procesos/subprocesos, límites de memoria y otros

In [1]:
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client.cluster

Si no está en jupyterlab y utiliza el `dask-labextension`, asegúrese de hacer clic en el enlace`Dashboard` para abrir el tablero de diagnóstico.

## Ejecutando con el cliente distribuido

Considere un cálculo trivial, como el que hemos usado antes, donde hemos agregado declaraciones de sueño para simular el trabajo real que se está realizando.

In [2]:
from dask import delayed
import time

def inc(x):
    time.sleep(5)
    return x + 1

def dec(x):
    time.sleep(3)
    return x - 1

def add(x, y):
    time.sleep(7)
    return x + y

De forma predeterminada, la creación de un `Cliente` lo convierte en el planificador predeterminado. Cualquier llamada a `.compute` usará el clúster al que está adjunto su `cliente`, a menos que especifique lo contrario, como se indicó anteriormente.


In [3]:
x = delayed(inc)(1)
y = delayed(dec)(2)
total = delayed(add)(x, y)
total.compute()

3

Las tareas aparecerán en la interfaz de usuario web a medida que el clúster las procese y, finalmente, se imprimirá un resultado como salida de la celda anterior. Tenga en cuenta que el kernel está bloqueado mientras espera el resultado. El gráfico de bloques de tareas resultante podría tener un aspecto similar al siguiente. Al pasar el cursor sobre cada bloque, se indica con qué función se relaciona y cuánto tiempo tardó en ejecutarse. ![esto](../images/tasks.png)

También puede ver una versión simplificada del gráfico que se está ejecutando en el panel Gráfico del tablero, siempre que el cálculo esté en curso.

Regresemos al cálculo de vuelos de antes y veamos qué sucede en el tablero (es posible que desee tener el portátil y el tablero uno al lado del otro). ¿Cómo funcionó esto en comparación con antes?

In [6]:
%time largest_delay.compute()

CPU times: user 1.45 s, sys: 193 ms, total: 1.64 s
Wall time: 8.41 s


10.351298909519874

En este caso particular, esto debería ser tan rápido o más rápido que en el mejor de los casos, `threaded` arriba. ¿Qué supones que es esto? Debe comenzar su lectura [aquí](https://distributed.dask.org/en/latest/index.html#architecture) y, en particular, tenga en cuenta que el programador distribuido fue una reescritura completa con más inteligencia sobre el intercambio de resultados intermedios y qué tareas se ejecutan en qué trabajador. Esto dará como resultado un mejor rendimiento en *algunos* casos, pero una latencia y una sobrecarga aún mayores en comparación con el programador de subprocesos, por lo que habrá casos raros en los que se desempeñe peor. Afortunadamente, el panel ahora nos brinda mucha más [información de diagnóstico](https://distributed.dask.org/en/latest/diagnosing-performance.html). Mire la página Perfil del tablero para averiguar qué es lo que toma la mayor fracción de tiempo de CPU para el cálculo que acabamos de realizar.

Si todo lo que quiere hacer es ejecutar cálculos creados usando retrasos, o ejecutar cálculos basados en las colecciones de datos de nivel superior, entonces eso es todo lo que necesita saber para escalar su trabajo a escala de clúster. Sin embargo, hay más detalles que conocer sobre el programador distribuido que ayudará con el uso eficiente. Consulte el capítulo Distribuido, Avanzado.

### Ejercicio

Ejecute los siguientes cálculos mientras mira la página de diagnóstico. En cada caso, ¿qué está tomando más tiempo?

In [7]:
# Number of flights
_ = len(df)

In [8]:
# Number of non-cancelled flights
_ = len(df[~df.Cancelled])

In [ ]:
# Number of non-cancelled flights per-airport
_ = df[~df.Cancelled].groupby('Origin').Origin.count().compute()

In [9]:
# Average departure delay from each airport?
_ = df[~df.Cancelled].groupby('Origin').DepDelay.mean().compute()

In [10]:
# Average departure delay per day-of-week
_ = df.groupby(df.Date.dt.dayofweek).DepDelay.mean().compute()

In [11]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
